# PersonalizedRouter - Inference

This notebook demonstrates how to use a trained **PersonalizedRouter** for inference.

## Overview

PersonalizedRouter supports both:
- **Single query routing** with `route_single()`
- **Batch routing** with `route_batch()`

The router considers user features for personalized routing.

## 1. Environment Setup

In [ ]:
import os
import sys
from pathlib import Path

PROJECT_ROOT = Path(os.getcwd()).parent.parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

os.chdir(PROJECT_ROOT)

In [ ]:
import torch
from llmrouter.models.personalizedrouter import PersonalizedRouter
from llmrouter.utils import setup_environment
import yaml

setup_environment()

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

## 2. Load Trained Router

In [ ]:
CONFIG_PATH = "configs/model_config_train/personalizedrouter.yaml"

with open(CONFIG_PATH, 'r') as f:
    config = yaml.safe_load(f)

router = PersonalizedRouter(yaml_path=CONFIG_PATH)
print(f"Router loaded with {len(router.llm_data)} LLM candidates")
print(f"Number of users: {router.num_users}")
print(f"Number of tasks: {router.num_task}")

## 3. Single Query Routing

Route a single query with optional user_id for personalization.

In [ ]:
# Example queries
EXAMPLE_QUERIES = [
    {"query": "What is the capital of France?", "user_id": 0},
    {"query": "Solve the equation: 2x + 5 = 15", "user_id": 1},
    {"query": "Write a Python function to check if a number is prime.", "user_id": 2},
    {"query": "Explain quantum computing in simple terms.", "user_id": 0},
]

print("Routing Results:")
print("=" * 60)

for i, query in enumerate(EXAMPLE_QUERIES, 1):
    result = router.route_single(query)
    print(f"{i}. Query: {query['query'][:50]}...")
    print(f"   User ID: {query.get('user_id', 'N/A')}")
    print(f"   Routed to: {result['model_name']}")
    print()

## 4. Batch Routing

Route multiple queries efficiently using batch inference.

In [ ]:
# Prepare batch queries
batch_queries = [
    {"query": "What is the capital of France?", "user_id": 0},
    {"query": "Who wrote Romeo and Juliet?", "user_id": 1},
    {"query": "How does photosynthesis work?", "user_id": 2},
    {"query": "What is the speed of light?", "user_id": 0},
    {"query": "Explain the theory of relativity.", "user_id": 1},
]

print("Batch Routing Results:")
print("=" * 60)

# Route batch
batch_results = router.route_batch(batch=batch_queries)

for i, result in enumerate(batch_results, 1):
    print(f"{i}. Query: {result.get('query', '')[:45]}...")
    print(f"   User ID: {result.get('user_id', 'N/A')}")
    print(f"   Routed to: {result['model_name']}")
    if 'response' in result and result['response']:
        print(f"   Response: {result['response'][:80]}...")
    print()

## 5. File-Based Inference

Load queries from a file and save results.

In [ ]:
import json

# Load queries from a JSONL file
def load_queries_from_file(file_path):
    """Load queries from a JSONL file."""
    queries = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                queries.append(json.loads(line))
    return queries

# Save results to a JSONL file
def save_results_to_file(results, output_path):
    """Save routing results to a JSONL file."""
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with open(output_path, 'w', encoding='utf-8') as f:
        for result in results:
            f.write(json.dumps(result, ensure_ascii=False) + '\n')
    print(f"Results saved to: {output_path}")

# Example: Load from default query file
QUERY_FILE = "data/example_data/query_data/default_query_test.jsonl"
OUTPUT_FILE = "outputs/personalizedrouter_results.jsonl"

if os.path.exists(QUERY_FILE):
    # Load queries
    file_queries = load_queries_from_file(QUERY_FILE)
    print(f"Loaded {len(file_queries)} queries from: {QUERY_FILE}")
    
    # Add user_id if missing (cycle through users)
    num_users = int(getattr(router, "num_users", 1) or 1)
    for i, query in enumerate(file_queries):
        query.setdefault('user_id', i % num_users)
    
    # Route queries (limit to first 10 for demo)
    file_results = router.route_batch(batch=file_queries[:10])
    print(f"Routed {len(file_results)} queries")
    
    # Save results
    save_results_to_file(file_results, OUTPUT_FILE)
    
    # Show sample results
    print(f"\nSample results:")
    for i, result in enumerate(file_results[:3], 1):
        query_text = result.get('query', '')[:40]
        user_id = result.get('user_id', 'N/A')
        print(f"  {i}. {query_text}... (user {user_id}) -> {result['model_name']}")
else:
    print(f"Query file not found: {QUERY_FILE}")
    print("Create a JSONL file with format: {\"query\": \"Your question\", \"user_id\": 0}")

## 6. Personalized Routing Analysis

Analyze how routing differs across users.

In [ ]:
# Test the same query with different users
TEST_QUERY = {"query": "What is the capital of France?"}

print(f"Test Query: {TEST_QUERY['query']}")
print("=" * 60)
print("Routing results for different users:")
print()

user_routing_results = {}
for user_id in range(min(5, router.num_users)):
    test_query = TEST_QUERY.copy()
    test_query['user_id'] = user_id
    result = router.route_single(test_query)
    user_routing_results[user_id] = result['model_name']
    print(f"User {user_id}: {result['model_name']}")

print()
if len(set(user_routing_results.values())) > 1:
    print("Note: Routing differs across users - personalization is working!")
else:
    print("Note: All users routed to the same model.")
    print("This is expected if training data doesn't show strong user preferences.")

## Summary

This notebook demonstrated:
1. Loading a trained PersonalizedRouter
2. Single query routing with user personalization
3. Batch routing for efficient inference
4. File-based inference pipeline
5. Personalized routing analysis across users

PersonalizedRouter is effective for:
- User-specific routing preferences
- Multi-user environments
- Learning different routing strategies for different users